Importing the libraries needed for the project, and reading in the first dataset (boardgames_ranks.csv)

In [281]:
import pandas as pd
import requests
import matplotlib.pyplot as plt
import numpy as np
import xml.etree.ElementTree as ET
import time
from bs4 import BeautifulSoup
import re

df_board_game_rankings = pd.read_csv('.\\.venv\\data\\boardgames_ranks.csv')
df_board_game_rankings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 158945 entries, 0 to 158944
Data columns (total 16 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   id                   158945 non-null  int64  
 1   name                 158945 non-null  object 
 2   yearpublished        158945 non-null  int64  
 3   rank                 158945 non-null  int64  
 4   bayesaverage         158945 non-null  float64
 5   average              158945 non-null  float64
 6   usersrated           158945 non-null  int64  
 7   is_expansion         158945 non-null  int64  
 8   abstracts_rank       1424 non-null    float64
 9   cgs_rank             362 non-null     float64
 10  childrensgames_rank  1073 non-null    float64
 11  familygames_rank     3314 non-null    float64
 12  partygames_rank      912 non-null     float64
 13  strategygames_rank   2992 non-null    float64
 14  thematic_rank        1671 non-null    float64
 15  wargames_rank    

Filter out games with rank of 0, as we only want games that have been ranked.

In [282]:
df_bgr2 = pd.read_csv('.\\.venv\\data\\boardgames_ranks.csv')
zerorank = df_bgr2[(df_bgr2['rank'] == 0)].index
df_bgr_nozero = pd.DataFrame(df_bgr2.drop(zerorank, inplace = True))
df_bgr2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27288 entries, 0 to 27287
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   27288 non-null  int64  
 1   name                 27288 non-null  object 
 2   yearpublished        27288 non-null  int64  
 3   rank                 27288 non-null  int64  
 4   bayesaverage         27288 non-null  float64
 5   average              27288 non-null  float64
 6   usersrated           27288 non-null  int64  
 7   is_expansion         27288 non-null  int64  
 8   abstracts_rank       1424 non-null   float64
 9   cgs_rank             362 non-null    float64
 10  childrensgames_rank  1073 non-null   float64
 11  familygames_rank     3314 non-null   float64
 12  partygames_rank      912 non-null    float64
 13  strategygames_rank   2992 non-null   float64
 14  thematic_rank        1671 non-null   float64
 15  wargames_rank        4311 non-null  

Pulling only the top 250 overall rank from boardgames_ranks.csv

In [283]:
df_bgr_top = pd.DataFrame(df_bgr2.loc[(df_bgr2['rank'] >=1) & (df_bgr2['rank'] <=250)])
df_bgr_top.head(20)

,id,name,yearpublished,rank,bayesaverage,average,usersrated,is_expansion,abstracts_rank,cgs_rank,childrensgames_rank,familygames_rank,partygames_rank,strategygames_rank,thematic_rank,wargames_rank
0,224517,Brass: Birmingham,2018,1,8.41277,8.59213,48001,0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
1,161936,Pandemic Legacy: Season 1,2015,2,8.37571,8.52498,54148,0,NaN,NaN,NaN,NaN,NaN,2.0,1.0,NaN
2,174430,Gloomhaven,2017,3,8.34398,8.57972,63128,0,NaN,NaN,NaN,NaN,NaN,4.0,2.0,NaN
3,342942,Ark Nova,2021,4,8.33959,8.53505,46382,0,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN
4,233078,Twilight Imperium: Fourth Edition,2017,5,8.23662,8.59240,24556,0,NaN,NaN,NaN,NaN,NaN,5.0,3.0,NaN
5,316554,Dune: Imperium,2020,6,8.23137,8.43403,47922,0,NaN,NaN,NaN,NaN,NaN,6.0,NaN,NaN
6,167791,Terraforming Mars,2016,7,8.20734,8.35483,101433,0,NaN,NaN,NaN,NaN,NaN,7.0,NaN,NaN
7,115746,War of the Ring: Second Edition,2011,8,8.19064,8.54570,22008,0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,1.0
8,187645,Star Wars: Rebellion,2016,9,8.16964,8.41835,33163,0,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN
9,291457,Gloomhaven: Jaws of the Lion,2020,10,8.15191,8.42294,35486,0,NaN,NaN,NaN,NaN,NaN,9.0,7.0,NaN


Replacing NaN values in the DataFrame with '0'

In [284]:
df_bgr_top.fillna(0, inplace=True)


Converting rank fields to int.

In [287]:
convert_dict = {'abstracts_rank': int,
                'cgs_rank': int,
                'childrensgames_rank': int,
                'familygames_rank': int,
                'partygames_rank': int,
                'strategygames_rank': int,
                'thematic_rank': int,
                'wargames_rank': int}
df_bgr_top = df_bgr_top.astype(convert_dict)
print(df_bgr_top.dtypes)

id                       int64
name                    object
yearpublished            int64
rank                     int64
bayesaverage           float64
average                float64
usersrated               int64
is_expansion             int64
abstracts_rank           int64
cgs_rank                 int64
childrensgames_rank      int64
familygames_rank         int64
partygames_rank          int64
strategygames_rank       int64
thematic_rank            int64
wargames_rank            int64
dtype: object


In [288]:
df_bgr_top.info()

<class 'pandas.core.frame.DataFrame'>
Index: 250 entries, 0 to 249
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   250 non-null    int64  
 1   name                 250 non-null    object 
 2   yearpublished        250 non-null    int64  
 3   rank                 250 non-null    int64  
 4   bayesaverage         250 non-null    float64
 5   average              250 non-null    float64
 6   usersrated           250 non-null    int64  
 7   is_expansion         250 non-null    int64  
 8   abstracts_rank       250 non-null    int64  
 9   cgs_rank             250 non-null    int64  
 10  childrensgames_rank  250 non-null    int64  
 11  familygames_rank     250 non-null    int64  
 12  partygames_rank      250 non-null    int64  
 13  strategygames_rank   250 non-null    int64  
 14  thematic_rank        250 non-null    int64  
 15  wargames_rank        250 non-null    int64  


Rounding averages to 2 digits.

In [289]:
df_bgr_top.bayesaverage = df_bgr2.bayesaverage.round(2)
df_bgr_top.average = df_bgr2.average.round(2)
df_bgr_top.head(20)

,id,name,yearpublished,rank,bayesaverage,average,usersrated,is_expansion,abstracts_rank,cgs_rank,childrensgames_rank,familygames_rank,partygames_rank,strategygames_rank,thematic_rank,wargames_rank
0,224517,Brass: Birmingham,2018,1,8.41,8.59,48001,0,0,0,0,0,0,1,0,0
1,161936,Pandemic Legacy: Season 1,2015,2,8.38,8.52,54148,0,0,0,0,0,0,2,1,0
2,174430,Gloomhaven,2017,3,8.34,8.58,63128,0,0,0,0,0,0,4,2,0
3,342942,Ark Nova,2021,4,8.34,8.54,46382,0,0,0,0,0,0,3,0,0
4,233078,Twilight Imperium: Fourth Edition,2017,5,8.24,8.59,24556,0,0,0,0,0,0,5,3,0
5,316554,Dune: Imperium,2020,6,8.23,8.43,47922,0,0,0,0,0,0,6,0,0
6,167791,Terraforming Mars,2016,7,8.21,8.35,101433,0,0,0,0,0,0,7,0,0
7,115746,War of the Ring: Second Edition,2011,8,8.19,8.55,22008,0,0,0,0,0,0,0,4,1
8,187645,Star Wars: Rebellion,2016,9,8.17,8.42,33163,0,0,0,0,0,0,0,5,0
9,291457,Gloomhaven: Jaws of the Lion,2020,10,8.15,8.42,35486,0,0,0,0,0,0,9,7,0


Renaming Columns

In [290]:
df_bgr_top.rename(columns={'id': 'BGG_ID', 'name': 'GAME_NAME', 'yearpublished': 'PUBLISH_YR', 'rank': 'OVERALL_RANK', 'bayesaverage': 'BAYES_AVG',
                           'average': 'AVG_RATING','usersrated': 'USER_RATING', 'is_expansion': 'EXPANSION_IND', 'abstracts_rank': 'ABSTRACTS_RANK',
                           'cgs_rank': 'CGS_RANK', 'childrensgames_rank': 'CHILDREN_GAME_RANK', 'familygames_rank': 'FAMILY_RANK', 'partygames_rank': 'PARTY_RANK',
                           'strategygames_rank': 'STRATEGY_RANK', 'thematic_rank': 'THEMATIC_RANK', 'wargames_rank': 'WARGAME_RANK'}, inplace=True)

df_bgr_top.info()

<class 'pandas.core.frame.DataFrame'>
Index: 250 entries, 0 to 249
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   BGG_ID              250 non-null    int64  
 1   GAME_NAME           250 non-null    object 
 2   PUBLISH_YR          250 non-null    int64  
 3   OVERALL_RANK        250 non-null    int64  
 4   BAYES_AVG           250 non-null    float64
 5   AVG_RATING          250 non-null    float64
 6   USER_RATING         250 non-null    int64  
 7   EXPANSION_IND       250 non-null    int64  
 8   ABSTRACTS_RANK      250 non-null    int64  
 9   CGS_RANK            250 non-null    int64  
 10  CHILDREN_GAME_RANK  250 non-null    int64  
 11  FAMILY_RANK         250 non-null    int64  
 12  PARTY_RANK          250 non-null    int64  
 13  STRATEGY_RANK       250 non-null    int64  
 14  THEMATIC_RANK       250 non-null    int64  
 15  WARGAME_RANK        250 non-null    int64  
dtypes: float64(2)

Outputting to csv

Extract BGG IDs from csv into list to feed into API.

In [ ]:
bg_ids = df_bgr_top[str('BGG_ID')].values.tolist()
print(bg_ids)




[224517, 161936, 174430, 342942, 233078, 316554, 167791, 115746, 187645, 291457, 162886, 220308, 12333, 182028, 193738, 84876, 169786, 246900, 173346, 28720, 167355, 266507, 177736, 124361, 295770, 341169, 205637, 312484, 266192, 120677, 237182, 192135, 164928, 397598, 96848, 251247, 324856, 199792, 183394, 366013, 285774, 175914, 256960, 247763, 284378, 3076, 521, 295947, 102794, 185343, 184267, 314040, 170216, 31260, 255984, 161533, 253344, 221107, 373106, 251661, 205059, 231733, 321608, 126163, 182874, 2651, 276025, 244521, 216132, 35677, 266810, 125153, 164153, 124742, 209010, 284083, 55690, 200680, 230802, 28143, 157354, 383179, 72125, 201808, 191189, 159675, 229853, 110327, 317985, 25613, 93, 62219, 365717, 68448, 121921, 171623, 256916, 279537, 332772, 366161, 291453, 225694, 264220, 155821, 37111, 236457, 163068, 322289, 310873, 122515, 170042, 42, 40834, 364073, 18602, 203993, 73439, 146021, 271055, 172386, 277659, 269385, 217372, 371942, 180263, 12493, 163412, 144733, 102680,

Larger Scale Test

In [316]:
df_bgr_30 = pd.DataFrame(df_bgr_top.loc[(df_bgr_top['OVERALL_RANK'] >=1) & (df_bgr_top['OVERALL_RANK'] <=250)])
bg_ids_30 = df_bgr_30['BGG_ID'].values.tolist()
##bgidtest30 = [i for (i,x) in enumerate(bg_ids_30) if x%7 == 0]
for id in bg_ids_30:
    if id%4 == 0:
        print(id)

161936
220308
182028
84876
246900
28720
177736
312484
266192
164928
96848
324856
199792
256960
3076
314040
170216
31260
255984
253344
321608
216132
200680
201808
68448
256916
332772
264220
163068
217372
163412
102680
12
205896
310100
175640
198928
196340
178900
240980
266524
254640
118048
262712
147020
274364
390092
189932
271324
268864
14996
146652
283948
265188
294484
350184
188920
271896
227224
282524
338960
188
123260
150376
148228
256680


In [295]:
API_base_string = 'https://boardgamegeek.com/xmlapi2/thing?id='
bgg_id_ls = []
des_id_ls = []
des_name_ls = []
for id in bg_ids_30:
    api_rec_30 = requests.get(API_base_string + str(id))
    api_data_30 = api_rec_30.content
    root = ET.fromstring(api_data_30)
    if id%4 == 0: 
      time.sleep(6)
      for item in root:
        for link in item.findall('link'):
          if(link.get('type') == 'boardgamedesigner'):
            des_id_ls.append(link.get('id'))
            des_name_ls.append(link.get('value'))
            bgg_id_ls.append(item.get('id'))
    else:
      for item in root:
        for link in item.findall('link'):
          if(link.get('type') == 'boardgamedesigner'):
            des_id_ls.append(link.get('id'))
            des_name_ls.append(link.get('value'))
            bgg_id_ls.append(item.get('id'))
         
des_id_ser = pd.Series(des_id_ls)
des_name_ser = pd.Series(des_name_ls)
bgg_id_ser = pd.Series(bgg_id_ls)
desv3_frame = {'BGG_ID': bgg_id_ser, 'DESIGNER_ID': des_id_ser, 'DESIGNER_NAME': des_name_ser}
df_designersv3 = pd.DataFrame(desv3_frame)
df_designersv3



,BGG_ID,DESIGNER_ID,DESIGNER_NAME
0,224517,32887,Gavan Brown
1,224517,32943,Matt Tolman
2,224517,6,Martin Wallace
3,161936,442,Rob Daviau
4,161936,378,Matt Leacock
5,174430,69802,Isaac Childres
6,342942,138517,Mathias Wigge
7,233078,96049,Dane Beltrami
8,233078,6651,Corey Konieczka
9,233078,21,Christian T. Petersen


Full Run of 250 BGGIDS, pulling designer information

In [234]:
API_base_string = 'https://boardgamegeek.com/xmlapi2/thing?id='
bgg_id_ls = []
des_id_ls = []
des_name_ls = []
for id in bg_ids:
    api_rec_full = requests.get(API_base_string + str(id))
    api_data_full = api_rec_full.content
    root = ET.fromstring(api_data_full)
    if id%2 == 0: 
      time.sleep(15)
      for item in root:
        for link in item.findall('link'):
          if(link.get('type') == 'boardgamedesigner'):
            des_id_ls.append(link.get('id'))
            des_name_ls.append(link.get('value'))
            bgg_id_ls.append(item.get('id'))
    else:
      for item in root:
        for link in item.findall('link'):
          if(link.get('type') == 'boardgamedesigner'):
            des_id_ls.append(link.get('id'))
            des_name_ls.append(link.get('value'))
            bgg_id_ls.append(item.get('id'))
         
des_id_ser = pd.Series(des_id_ls)
des_name_ser = pd.Series(des_name_ls)
bgg_id_ser = pd.Series(bgg_id_ls)
desv3_frame = {'BGG_ID': bgg_id_ser, 'Designer_ID': des_id_ser, 'Designer_Name': des_name_ser}
df_designersv3 = pd.DataFrame(desv3_frame)
df_designersv3.to_csv('.\\.venv\\data\\bgg_designer_api_pull.csv', index=False)
df_designersv3

,BGG_ID,Designer_ID,Designer_Name
0,224517,32887,Gavan Brown
1,224517,32943,Matt Tolman
2,224517,6,Martin Wallace
3,161936,442,Rob Daviau
4,161936,378,Matt Leacock
...,...,...,...
383,283155,117880,Kevin Russ
384,54998,1727,Bruno Cathala
385,54998,4337,Ludovic Maublanc
386,221194,12679,Jonathan Gilmour-Long


Need to Tweak to pull Mechanic information by BGG_ID

In [296]:
mech_id_ls = []
mech_name_ls = []
mech_bggid_ls = []

for id in bg_ids_30:
    api_rec = requests.get(API_base_string + str(id))
    api_data = api_rec.content
    root = ET.fromstring(api_data)
    if id%7 == 0: 
      time.sleep(5)
      for item in root:
        for link in item.findall('link'):
          if(link.get('type') == 'boardgamemechanic'):
            mech_id_ls.append(link.get('id'))
            mech_name_ls.append(link.get('value'))
            mech_bggid_ls.append(item.get('id'))
    else:
      for item in root:
        for link in item.findall('link'):
           if(link.get('type') == 'boardgamemechanic'):
            mech_id_ls.append(link.get('id'))
            mech_name_ls.append(link.get('value'))
            mech_bggid_ls.append(item.get('id'))
         
mech_bggid_ser = pd.Series(mech_bggid_ls)
mech_id_ser = pd.Series(mech_id_ls)
mech_name_ser = pd.Series(mech_name_ls)
mechv3_frame = {'BGG_ID': mech_bggid_ser, 'MECHANIC_ID': mech_id_ser, 'MECHANIC_NAME': mech_name_ser}
df_mechv3 = pd.DataFrame(mechv3_frame)
df_mechv3.head(50)

,BGG_ID,MECHANIC_ID,MECHANIC_NAME
0,224517,2040,Hand Management
1,224517,2902,Income
2,224517,2904,Loans
3,224517,2900,Market
4,224517,3099,Multi-Use Cards
5,224517,2081,Network and Route Building
6,224517,3100,Tags
7,224517,2849,Tech Trees / Tech Tracks
8,224517,2826,Turn Order: Stat-Based
9,224517,2897,Variable Set-up


In [297]:
df_mechv3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 387 entries, 0 to 386
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   BGG_ID         387 non-null    object
 1   MECHANIC_ID    387 non-null    object
 2   MECHANIC_NAME  387 non-null    object
dtypes: object(3)
memory usage: 9.2+ KB


In [304]:
mech_convert_dict = {'BGG_ID': int,
                'MECHANIC_ID': object}
df_mechv3 = df_mechv3.astype(mech_convert_dict)
df_mechv3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 387 entries, 0 to 386
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   BGG_ID         387 non-null    int64 
 1   MECHANIC_ID    387 non-null    object
 2   MECHANIC_NAME  387 non-null    object
dtypes: int64(1), object(2)
memory usage: 9.2+ KB


In [305]:
df_mech_merge = pd.merge(df_bgr_top, df_mechv3, on='BGG_ID',how='left')

df_gameplusmech = pd.DataFrame(df_mech_merge)
df_gameplusmech.head(15)

,BGG_ID,GAME_NAME,PUBLISH_YR,OVERALL_RANK,BAYES_AVG,AVG_RATING,USER_RATING,EXPANSION_IND,ABSTRACTS_RANK,CGS_RANK,CHILDREN_GAME_RANK,FAMILY_RANK,PARTY_RANK,STRATEGY_RANK,THEMATIC_RANK,WARGAME_RANK,MECHANIC_ID,MECHANIC_NAME
0,224517,Brass: Birmingham,2018,1,8.41,8.59,48001,0,0,0,0,0,0,1,0,0,2040,Hand Management
1,224517,Brass: Birmingham,2018,1,8.41,8.59,48001,0,0,0,0,0,0,1,0,0,2902,Income
2,224517,Brass: Birmingham,2018,1,8.41,8.59,48001,0,0,0,0,0,0,1,0,0,2904,Loans
3,224517,Brass: Birmingham,2018,1,8.41,8.59,48001,0,0,0,0,0,0,1,0,0,2900,Market
4,224517,Brass: Birmingham,2018,1,8.41,8.59,48001,0,0,0,0,0,0,1,0,0,3099,Multi-Use Cards
5,224517,Brass: Birmingham,2018,1,8.41,8.59,48001,0,0,0,0,0,0,1,0,0,2081,Network and Route Building
6,224517,Brass: Birmingham,2018,1,8.41,8.59,48001,0,0,0,0,0,0,1,0,0,3100,Tags
7,224517,Brass: Birmingham,2018,1,8.41,8.59,48001,0,0,0,0,0,0,1,0,0,2849,Tech Trees / Tech Tracks
8,224517,Brass: Birmingham,2018,1,8.41,8.59,48001,0,0,0,0,0,0,1,0,0,2826,Turn Order: Stat-Based
9,224517,Brass: Birmingham,2018,1,8.41,8.59,48001,0,0,0,0,0,0,1,0,0,2897,Variable Set-up


Pivot experiment


*******

In [385]:
df_gamemech_pivot = df_gameplusmech.pivot_table(values='BGG_ID', index=('MECHANIC_NAME'),columns=None, aggfunc='count')
flat_df_gamemech_pivot = df_gamemech_pivot.reset_index()
name_dict = {'MECHANIC_NAME': 'MECHANIC_NAME', 'BGG_ID': 'MECHANIC_COUNT'}

flat_df_gamemech_pivot.rename(columns=name_dict,inplace=True)
flat_df_gamemech_pivot.sort_values('MECHANIC_COUNT',ascending=False, inplace=True)

flat_df_gamemech_pivot.head(95)

,MECHANIC_NAME,MECHANIC_COUNT
32,Hand Management,18
88,Variable Player Powers,18
89,Variable Set-up,17
71,Solo / Solitaire Game,15
75,Tags,12
...,...,...
82,Traitor Game,1
87,Variable Phase Order,1
91,Voting,1
93,"Worker Placement, Different Worker Types",1


In [180]:
response_API_test = requests.get('https://boardgamegeek.com/xmlapi2/thing?id=224517')
api_data = response_API_test.content

##json_bgame_data = xmltodict.parse(api_data)
soup = BeautifulSoup(api_data, 'html.parser')
print(soup.prettify())

<?xml version="1.0" encoding="utf-8"?>
<items termsofuse="https://boardgamegeek.com/xmlapi/termsofuse">
 <item id="224517" type="boardgame">
  <thumbnail>
   https://cf.geekdo-images.com/x3zxjr-Vw5iU4yDPg70Jgw__thumb/img/o18rjEemoWaVru9Y2TyPwuIaRfE=/fit-in/200x150/filters:strip_icc()/pic3490053.jpg
  </thumbnail>
  <image/>
  https://cf.geekdo-images.com/x3zxjr-Vw5iU4yDPg70Jgw__original/img/FpyxH41Y6_ROoePAilPNEhXnzO8=/0x0/filters:format(jpeg)/pic3490053.jpg
  <name sortindex="1" type="primary" value="Brass: Birmingham">
  </name>
  <name sortindex="1" type="alternate" value="Brass. Бирмингем">
  </name>
  <name sortindex="1" type="alternate" value="Brass. Бірмінгем">
  </name>
  <name sortindex="1" type="alternate" value="ブラス：バーミンガム">
  </name>
  <name sortindex="1" type="alternate" value="工业革命：伯明翰(Chinese edition) (2018)">
  </name>
  <name sortindex="1" type="alternate" value="工業革命：伯明翰">
  </name>
  <name sortindex="1" type="alternate" value="브라스: 버밍엄">
  </name>
  <description>
   

C:\Program Files\Python312\Lib\html\parser.py:171: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  k = self.parse_starttag(i)


In [395]:
##2
df_bgdesigner_loc = pd.read_csv('.\\.venv\\data\\BGG_Designer_Location_Key.csv')
df_bgdesigner_loc.fillna('UNK', inplace=True)
df_bgdesigner_loc.head()

,BGG_ID,DESIGNER_ID,DESIGNER_NAME,DESIGNER_POB_CITY,DESIGNER_POB_ST-PROV,DESIGNER_POB_CTRY
0,224517,32887,Gavan Brown,Calgary,AB,CA
1,224517,32943,Matt Tolman,Calgary,AB,CA
2,224517,6,Martin Wallace,Manchester,UNK,UK
3,161936,442,Rob Daviau,Boston,MA,US
4,161936,378,Matt Leacock,UNK,CA,US


Export to CSV

In [313]:
df_bgdesigner_loc.to_csv('.\\.venv\\data\\top_designer_loc.csv', index=False)

Merging Designers DF with BGG Ranks on BGG_ID

In [408]:
df_design_merge = pd.merge(df_bgr_top, df_bgdesigner_loc, on='BGG_ID',how='left')
df_game_des = pd.DataFrame(df_design_merge)
##df_game_des.fillna(0, inplace=True)
##des_convert_dict = {'DESIGNER_ID': int,
  ##                  'DESIGNER_NAME': int,
    ##               }
##df_game_des = df_game_des.astype(des_convert_dict)

df_game_des

,BGG_ID,GAME_NAME,PUBLISH_YR,OVERALL_RANK,BAYES_AVG,AVG_RATING,USER_RATING,EXPANSION_IND,ABSTRACTS_RANK,CGS_RANK,...,FAMILY_RANK,PARTY_RANK,STRATEGY_RANK,THEMATIC_RANK,WARGAME_RANK,DESIGNER_ID,DESIGNER_NAME,DESIGNER_POB_CITY,DESIGNER_POB_ST-PROV,DESIGNER_POB_CTRY
0,224517,Brass: Birmingham,2018,1,8.41,8.59,48001,0,0,0,...,0,0,1,0,0,32887.0,Gavan Brown,Calgary,AB,CA
1,224517,Brass: Birmingham,2018,1,8.41,8.59,48001,0,0,0,...,0,0,1,0,0,32943.0,Matt Tolman,Calgary,AB,CA
2,224517,Brass: Birmingham,2018,1,8.41,8.59,48001,0,0,0,...,0,0,1,0,0,6.0,Martin Wallace,Manchester,UNK,UK
3,161936,Pandemic Legacy: Season 1,2015,2,8.38,8.52,54148,0,0,0,...,0,0,2,1,0,442.0,Rob Daviau,Boston,MA,US
4,161936,Pandemic Legacy: Season 1,2015,2,8.38,8.52,54148,0,0,0,...,0,0,2,1,0,378.0,Matt Leacock,UNK,CA,US
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319,269207,The Taverns of Tiefenthal,2019,246,7.27,7.58,14831,0,0,0,...,45,0,183,0,0,NaN,NaN,NaN,NaN,NaN
320,21050,Combat Commander: Europe,2006,247,7.27,7.95,5954,0,0,0,...,0,0,0,0,10,NaN,NaN,NaN,NaN,NaN
321,283155,Calico,2020,248,7.27,7.52,19699,0,8,0,...,46,0,0,0,0,117880.0,Kevin Russ,Portland,OR,US
322,54998,Cyclades,2009,249,7.27,7.51,21793,0,0,0,...,0,0,195,0,0,4337.0,Ludovic Maublanc,UNK,UNK,FR


Pivot Table

In [ ]:
df_gamedes_pivot = df_game_des.pivot_table(values='BGG_ID', index=('MECHANIC_NAME'),columns=None, aggfunc='count')
flat_df_gamemech_pivot = df_gamemech_pivot.reset_index()
name_dict = {'MECHANIC_NAME': 'MECHANIC_NAME', 'BGG_ID': 'MECHANIC_COUNT'}

flat_df_gamemech_pivot.rename(columns=name_dict,inplace=True)
flat_df_gamemech_pivot.sort_values('MECHANIC_COUNT',ascending=False, inplace=True)

flat_df_gamemech_pivot.head(95)